In [1]:
# import pandas
import pandas as pd

In [2]:
# set up folder link
folder_link = 'C:/Users/Michael/projects/kaggle/rossman/'

In [3]:
# create dataframes
train = pd.read_csv(folder_link + 'train.csv')
test = pd.read_csv(folder_link + 'test.csv')

C:\Users\Michael\Anaconda\lib\site-packages\pandas\io\parsers.py:1170: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [9]:
# remove closed stores from training data
train = train.loc[train.Sales > 0]

In [13]:
# remove NANs from test data
test.loc[test.Open.isnull(),'Open'] = 1

In [80]:
# group by store day of week open promo state holiday school holiday and reset index
means1 = train.groupby(['Store','DayOfWeek','Open','Promo','StateHoliday','SchoolHoliday']).Sales.mean()
means1 = means1.reset_index()

means2 = train.groupby(['Store','DayOfWeek','Open','Promo','StateHoliday']).Sales.mean()
means2 = means2.reset_index()

means3 = train.groupby(['Store','DayOfWeek','Open','Promo']).Sales.mean()
means3 = means3.reset_index()

means4 = train.groupby(['Store','DayOfWeek','Open']).Sales.mean()
means4 = means4.reset_index()

In [85]:
# merge dataframes
test_output1 = pd.merge(test, means1, on = ['Store','DayOfWeek','Open','Promo','StateHoliday','SchoolHoliday'],how='left')
test_output2 = pd.merge(test, means2, on = ['Store','DayOfWeek','Open','Promo','StateHoliday'],how='left')
test_output3 = pd.merge(test, means3, on = ['Store','DayOfWeek','Open','Promo'],how='left')
test_output4 = pd.merge(test, means4, on = ['Store','DayOfWeek','Open'],how='left')

In [86]:
# zero out sales for closed stores
test_output1.loc[test_output1.Open == 0,'Sales'] = 0

In [87]:
# update null values
test_output1.loc[test_output1.Sales.isnull(),'Sales'] = test_output2.loc[test_output1.Sales.isnull(),'Sales']
test_output1.loc[test_output1.Sales.isnull(),'Sales'] = test_output3.loc[test_output1.Sales.isnull(),'Sales']
test_output1.loc[test_output1.Sales.isnull(),'Sales'] = test_output4.loc[test_output1.Sales.isnull(),'Sales']

In [88]:
# check for nulls
test_output1.isnull().sum()

Id               0
Store            0
DayOfWeek        0
Date             0
Open             0
Promo            0
StateHoliday     0
SchoolHoliday    0
Sales            0
dtype: int64

In [90]:
# output to csv file
test_output1[['Id','Sales']].to_csv(folder_link + 'predictions_means.csv',index=False)